## State Population and Vaccination Data

In [1]:
# Import Pandas, Config Variables, and SQLalchemy
import pandas as pd
from config import endpoint, username, password
from sqlalchemy import create_engine

In [2]:
# Read CSVs for State Data
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("../data/state_covid_vaccinations_cleaned.csv")
state_latlons_df = pd.read_csv("../data/statelatlong.csv")

In [3]:
# State Populations
state_pop_df.head()

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [4]:
# State Vaccinations
state_vacc_df.head()

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,271550,175138,37120.0,23941.0,122463,16740,52380,7160
1,Alabama,963600,601159,19653.0,12261.0,467032,9525,133029,2713
2,Arkansas,630700,457386,20899.0,15156.0,330146,10940,124040,4110
3,American Samoa,29450,15382,52883.0,27621.0,10062,18068,5308,9532
4,Arizona,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107


In [5]:
state_latlons_df.head()

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [6]:
state_latlons_df = state_latlons_df.rename(columns={"State": "Abb", "City": "state"})

In [7]:
state_latlons_df.head()

,Abb,Latitude,Longitude,state
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California


In [8]:
# State Data Merged
state_merged_df = pd.merge(state_pop_df, state_vacc_df, how="inner", on="state")
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713
1,Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160
2,Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107
3,Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110
4,California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983


In [9]:
state_merged_df = pd.merge(state_merged_df, state_latlons_df, how="left", on="state")

In [10]:
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
0,Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713,AL,32.601011,-86.680736
1,Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160,AK,61.302501,-158.775020
2,Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107,AZ,34.168219,-111.930907
3,Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110,AR,34.751928,-92.131378
4,California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983,CA,37.271875,-119.270415


In [11]:
# Set State as the Index
state_info_df = state_merged_df.set_index("state") 
state_info_df.head()

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude
state,,,,,,,,,,,,
Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713,AL,32.601011,-86.680736
Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160,AK,61.302501,-158.775020
Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107,AZ,34.168219,-111.930907
Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110,AR,34.751928,-92.131378
California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983,CA,37.271875,-119.270415


In [12]:
# Add Empty Column for New Calculation
state_info_df["percent_vaccinated"] = ""
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713,AL,32.601011,-86.680736,
Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160,AK,61.302501,-158.775020,
Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107,AZ,34.168219,-111.930907,
Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110,AR,34.751928,-92.131378,
California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983,CA,37.271875,-119.270415,
Colorado,5758736,1210475,885581,21020.0,15378.0,609445,10583,271661,4717,CO,38.997934,-105.550567,
Connecticut,3565287,847200,654766,23762.0,18365.0,462710,12978,183146,5137,CT,41.518784,-72.757507,
Delaware,973764,188075,142103,19314.0,14593.0,111918,11493,29423,3022,DE,39.145251,-75.418921,
District of Columbia,705749,193900,116261,27474.0,16473.0,81699,11576,34388,4873,DC,38.899349,-77.014567,


In [13]:
# Calcuation for percent vaccinated by state
for index, row in state_info_df.iterrows():
    population = row["population"]
    vaccinated = row["total_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_info_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [14]:
# Check the data types
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     float64
 4   administered_per_100K             51 non-null     float64
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   Abb                               51 non-null     object 
 10  Latitude                          51 non-null     float64
 11  Longitude                         51 non-null     float64
 12  perc

In [15]:
# Convert percent vaccinated into a float
state_info_df = state_info_df.astype({"percent_vaccinated": 'float64'})
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     float64
 4   administered_per_100K             51 non-null     float64
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   Abb                               51 non-null     object 
 10  Latitude                          51 non-null     float64
 11  Longitude                         51 non-null     float64
 12  perc

In [16]:
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated
state,,,,,,,,,,,,,
Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713,AL,32.601011,-86.680736,12.26
Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160,AK,61.302501,-158.775020,23.94
Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107,AZ,34.168219,-111.930907,14.30
Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110,AR,34.751928,-92.131378,15.16
California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983,CA,37.271875,-119.270415,14.08
Colorado,5758736,1210475,885581,21020.0,15378.0,609445,10583,271661,4717,CO,38.997934,-105.550567,15.38
Connecticut,3565287,847200,654766,23762.0,18365.0,462710,12978,183146,5137,CT,41.518784,-72.757507,18.37
Delaware,973764,188075,142103,19314.0,14593.0,111918,11493,29423,3022,DE,39.145251,-75.418921,14.59
District of Columbia,705749,193900,116261,27474.0,16473.0,81699,11576,34388,4873,DC,38.899349,-77.014567,16.47


## NIH AWS Data / NYT

### Set up State Master DF

In [55]:
# Read NYT / NIH json 
nytimes_nih_covid_df = pd.read_json("../data/daily_covid_stats_by_state_nyt.json", lines=True)

In [56]:
# Get Data for Most Recent Provided Date
most_recent_date = "2021-02-12"
latest_nyt_stateData = nytimes_nih_covid_df[nytimes_nih_covid_df['date'] == f'{most_recent_date}']

In [57]:
latest_nyt_stateData.head(10)

,date,state,fips,cases,deaths
19044,2021-02-12,Alabama,1,478667,9180
19045,2021-02-12,Alaska,2,56008,271
19046,2021-02-12,Arizona,4,794224,14843
19047,2021-02-12,Arkansas,5,311608,5212
19048,2021-02-12,California,6,3469853,46435
19049,2021-02-12,Colorado,8,414171,5893
19050,2021-02-12,Connecticut,9,267337,7381
19051,2021-02-12,Delaware,10,82263,1269
19052,2021-02-12,District of Columbia,11,38670,976
19053,2021-02-12,Florida,12,1814414,28564


In [58]:
import datetime as dt

for index, row in nytimes_nih_covid_df.iterrows():
    date = row["date"].strftime('%Y-%m-%d')
    nytimes_nih_covid_df.at[index, "date"] = date

In [59]:
nytimes_nih_covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19099 entries, 0 to 19098
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    19099 non-null  object
 1   state   19099 non-null  object
 2   fips    19099 non-null  int64 
 3   cases   19099 non-null  int64 
 4   deaths  19099 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 746.2+ KB


In [60]:
# Merge with State Info DF that contains population and vaccinations
latest_data_merged = pd.merge(state_info_df, latest_nyt_stateData, on="state")
latest_data_merged.head(10)

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,Abb,Latitude,Longitude,percent_vaccinated,date,fips,cases,deaths
0,Alabama,4903185,963600,601159,19653.0,12261.0,467032,9525,133029,2713,AL,32.601011,-86.680736,12.26,2021-02-12,1,478667,9180
1,Alaska,731545,271550,175138,37120.0,23941.0,122463,16740,52380,7160,AK,61.302501,-158.775020,23.94,2021-02-12,2,56008,271
2,Arizona,7278717,1393475,1041021,19145.0,14302.0,811080,11143,226158,3107,AZ,34.168219,-111.930907,14.30,2021-02-12,4,794224,14843
3,Arkansas,3017804,630700,457386,20899.0,15156.0,330146,10940,124040,4110,AR,34.751928,-92.131378,15.16,2021-02-12,5,311608,5212
4,California,39512223,8051475,5562553,20377.0,14078.0,4345018,10997,1178577,2983,CA,37.271875,-119.270415,14.08,2021-02-12,6,3469853,46435
5,Colorado,5758736,1210475,885581,21020.0,15378.0,609445,10583,271661,4717,CO,38.997934,-105.550567,15.38,2021-02-12,8,414171,5893
6,Connecticut,3565287,847200,654766,23762.0,18365.0,462710,12978,183146,5137,CT,41.518784,-72.757507,18.37,2021-02-12,9,267337,7381
7,Delaware,973764,188075,142103,19314.0,14593.0,111918,11493,29423,3022,DE,39.145251,-75.418921,14.59,2021-02-12,10,82263,1269
8,District of Columbia,705749,193900,116261,27474.0,16473.0,81699,11576,34388,4873,DC,38.899349,-77.014567,16.47,2021-02-12,11,38670,976
9,Florida,21477737,4441375,3351430,20679.0,15604.0,2304435,10729,1028266,4788,FL,27.975728,-83.833017,15.60,2021-02-12,12,1814414,28564


In [61]:
# Create a Master Dataframe to start working with
state_master_df = latest_data_merged[['state', 'Abb', 'date', 'population', 'Latitude', 'Longitude', 'cases', 'deaths', 'total_distributed', 'total_administered', 'percent_vaccinated' ]]
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated
0,Alabama,AL,2021-02-12,4903185,32.601011,-86.680736,478667,9180,963600,601159,12.26
1,Alaska,AK,2021-02-12,731545,61.302501,-158.775020,56008,271,271550,175138,23.94
2,Arizona,AZ,2021-02-12,7278717,34.168219,-111.930907,794224,14843,1393475,1041021,14.30
3,Arkansas,AR,2021-02-12,3017804,34.751928,-92.131378,311608,5212,630700,457386,15.16
4,California,CA,2021-02-12,39512223,37.271875,-119.270415,3469853,46435,8051475,5562553,14.08
5,Colorado,CO,2021-02-12,5758736,38.997934,-105.550567,414171,5893,1210475,885581,15.38
6,Connecticut,CT,2021-02-12,3565287,41.518784,-72.757507,267337,7381,847200,654766,18.37
7,Delaware,DE,2021-02-12,973764,39.145251,-75.418921,82263,1269,188075,142103,14.59
8,District of Columbia,DC,2021-02-12,705749,38.899349,-77.014567,38670,976,193900,116261,16.47
9,Florida,FL,2021-02-12,21477737,27.975728,-83.833017,1814414,28564,4441375,3351430,15.60


In [62]:
# Create empty column to calculate estimated percent infected to date
state_master_df["est_percent_infected_to_date"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-12,4903185,32.601011,-86.680736,478667,9180,963600,601159,12.26,
1,Alaska,AK,2021-02-12,731545,61.302501,-158.775020,56008,271,271550,175138,23.94,
2,Arizona,AZ,2021-02-12,7278717,34.168219,-111.930907,794224,14843,1393475,1041021,14.30,
3,Arkansas,AR,2021-02-12,3017804,34.751928,-92.131378,311608,5212,630700,457386,15.16,
4,California,CA,2021-02-12,39512223,37.271875,-119.270415,3469853,46435,8051475,5562553,14.08,
5,Colorado,CO,2021-02-12,5758736,38.997934,-105.550567,414171,5893,1210475,885581,15.38,
6,Connecticut,CT,2021-02-12,3565287,41.518784,-72.757507,267337,7381,847200,654766,18.37,
7,Delaware,DE,2021-02-12,973764,39.145251,-75.418921,82263,1269,188075,142103,14.59,
8,District of Columbia,DC,2021-02-12,705749,38.899349,-77.014567,38670,976,193900,116261,16.47,
9,Florida,FL,2021-02-12,21477737,27.975728,-83.833017,1814414,28564,4441375,3351430,15.60,


In [63]:
# Calcuation for est percent infected by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    state_master_df.at[index, "est_percent_infected_to_date"] = percent_infected

In [64]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,AL,2021-02-12,4903185,32.601011,-86.680736,478667,9180,963600,601159,12.26,9.78
1,Alaska,AK,2021-02-12,731545,61.302501,-158.775020,56008,271,271550,175138,23.94,7.66
2,Arizona,AZ,2021-02-12,7278717,34.168219,-111.930907,794224,14843,1393475,1041021,14.30,10.93
3,Arkansas,AR,2021-02-12,3017804,34.751928,-92.131378,311608,5212,630700,457386,15.16,10.34
4,California,CA,2021-02-12,39512223,37.271875,-119.270415,3469853,46435,8051475,5562553,14.08,8.79
5,Colorado,CO,2021-02-12,5758736,38.997934,-105.550567,414171,5893,1210475,885581,15.38,7.2
6,Connecticut,CT,2021-02-12,3565287,41.518784,-72.757507,267337,7381,847200,654766,18.37,7.51
7,Delaware,DE,2021-02-12,973764,39.145251,-75.418921,82263,1269,188075,142103,14.59,8.46
8,District of Columbia,DC,2021-02-12,705749,38.899349,-77.014567,38670,976,193900,116261,16.47,5.49
9,Florida,FL,2021-02-12,21477737,27.975728,-83.833017,1814414,28564,4441375,3351430,15.60,8.46


In [65]:
# Add Estimated Percent Immune Column
state_master_df["est_percent_immune"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-12,4903185,32.601011,-86.680736,478667,9180,963600,601159,12.26,9.78,
1,Alaska,AK,2021-02-12,731545,61.302501,-158.775020,56008,271,271550,175138,23.94,7.66,
2,Arizona,AZ,2021-02-12,7278717,34.168219,-111.930907,794224,14843,1393475,1041021,14.30,10.93,
3,Arkansas,AR,2021-02-12,3017804,34.751928,-92.131378,311608,5212,630700,457386,15.16,10.34,
4,California,CA,2021-02-12,39512223,37.271875,-119.270415,3469853,46435,8051475,5562553,14.08,8.79,
5,Colorado,CO,2021-02-12,5758736,38.997934,-105.550567,414171,5893,1210475,885581,15.38,7.2,
6,Connecticut,CT,2021-02-12,3565287,41.518784,-72.757507,267337,7381,847200,654766,18.37,7.51,
7,Delaware,DE,2021-02-12,973764,39.145251,-75.418921,82263,1269,188075,142103,14.59,8.46,
8,District of Columbia,DC,2021-02-12,705749,38.899349,-77.014567,38670,976,193900,116261,16.47,5.49,
9,Florida,FL,2021-02-12,21477737,27.975728,-83.833017,1814414,28564,4441375,3351430,15.60,8.46,


In [66]:
# Calcuation for estimated immune by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    infected = row["cases"]
    deaths = row["deaths"]
    vaccinated = row["total_administered"]
    
    est_total_immune = infected + vaccinated
    est_population = population - deaths
    
    percent_immune = round((est_total_immune / est_population) * 100, 2)
    
    state_master_df.at[index, "est_percent_immune"] = percent_immune

In [67]:
state_master_df.head(10)

,state,Abb,date,population,Latitude,Longitude,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,AL,2021-02-12,4903185,32.601011,-86.680736,478667,9180,963600,601159,12.26,9.78,22.06
1,Alaska,AK,2021-02-12,731545,61.302501,-158.775020,56008,271,271550,175138,23.94,7.66,31.61
2,Arizona,AZ,2021-02-12,7278717,34.168219,-111.930907,794224,14843,1393475,1041021,14.30,10.93,25.27
3,Arkansas,AR,2021-02-12,3017804,34.751928,-92.131378,311608,5212,630700,457386,15.16,10.34,25.53
4,California,CA,2021-02-12,39512223,37.271875,-119.270415,3469853,46435,8051475,5562553,14.08,8.79,22.89
5,Colorado,CO,2021-02-12,5758736,38.997934,-105.550567,414171,5893,1210475,885581,15.38,7.2,22.59
6,Connecticut,CT,2021-02-12,3565287,41.518784,-72.757507,267337,7381,847200,654766,18.37,7.51,25.92
7,Delaware,DE,2021-02-12,973764,39.145251,-75.418921,82263,1269,188075,142103,14.59,8.46,23.07
8,District of Columbia,DC,2021-02-12,705749,38.899349,-77.014567,38670,976,193900,116261,16.47,5.49,21.98
9,Florida,FL,2021-02-12,21477737,27.975728,-83.833017,1814414,28564,4441375,3351430,15.60,8.46,24.08


### Set up Daily Cases DF

In [68]:
nyt_avg_daily_cases = nytimes_nih_covid_df
nyt_avg_daily_cases.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [69]:
nyt_avg_daily_cases = nyt_avg_daily_cases.groupby("date").sum()
nyt_avg_daily_cases.head()

,fips,cases,deaths
date,,,
2020-01-21,53,1,0
2020-01-22,53,1,0
2020-01-23,53,1,0
2020-01-24,70,2,0
2020-01-25,76,3,0


In [70]:
nyt_avg_daily_cases["daily_new_cases"] = ""

In [71]:
nyt_avg_daily_cases.head()

,fips,cases,deaths,daily_new_cases
date,,,,
2020-01-21,53,1,0,
2020-01-22,53,1,0,
2020-01-23,53,1,0,
2020-01-24,70,2,0,
2020-01-25,76,3,0,


In [72]:
previous_day = 0

In [73]:
for index, row in nyt_avg_daily_cases.iterrows():
    new_cases = row["cases"] - previous_day
    previous_day = row["cases"]
    nyt_avg_daily_cases.at[index, "daily_new_cases"] = new_cases

In [74]:
nyt_avg_daily_cases["date"] = ""

In [75]:
nyt_avg_daily_cases

,fips,cases,deaths,daily_new_cases,date
date,,,,,
2020-01-21,53,1,0,1,
2020-01-22,53,1,0,0,
2020-01-23,53,1,0,0,
2020-01-24,70,2,0,1,
2020-01-25,76,3,0,1,
...,...,...,...,...,...
2021-02-08,1762,27137074,464921,92739,
2021-02-09,1762,27233562,468091,96488,
2021-02-10,1762,27328455,471346,94893,


In [78]:
for index, row in nyt_avg_daily_cases.iterrows():
    date = index
    nyt_avg_daily_cases.at[index, "date"] = date

In [79]:
nyt_avg_daily_cases.reset_index(drop=True, inplace=True)

In [80]:
nyt_avg_daily_cases.head(10)

,fips,cases,deaths,daily_new_cases,date
0,53,1,0,1,2020-01-21
1,53,1,0,0,2020-01-22
2,53,1,0,0,2020-01-23
3,70,2,0,1,2020-01-24
4,76,3,0,1,2020-01-25
5,80,5,0,2,2020-01-26
6,80,5,0,0,2020-01-27
7,80,5,0,0,2020-01-28
8,80,5,0,0,2020-01-29
9,80,6,0,1,2020-01-30


### Set Up NYT Totals Master DF

In [81]:
nyt_master = pd.merge(nytimes_nih_covid_df, state_pop_df, how ="inner", on="state")
nyt_master.head()

,date,state,fips,cases,deaths,population
0,2020-01-21,Washington,53,1,0,7614893
1,2020-01-22,Washington,53,1,0,7614893
2,2020-01-23,Washington,53,1,0,7614893
3,2020-01-24,Washington,53,1,0,7614893
4,2020-01-25,Washington,53,1,0,7614893


In [82]:
nyt_master["est_percent_infected"] = ""
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,
1,2020-01-22,Washington,53,1,0,7614893,
2,2020-01-23,Washington,53,1,0,7614893,
3,2020-01-24,Washington,53,1,0,7614893,
4,2020-01-25,Washington,53,1,0,7614893,


In [83]:
# Calcuation for est percent infected by state
for index, row in nyt_master.iterrows():
    population = int(row["population"])
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    if infected == 0:
        percent_infected = 0
    else: 
        percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    nyt_master.at[index, "est_percent_infected"] = percent_infected

In [84]:
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,0
1,2020-01-22,Washington,53,1,0,7614893,0
2,2020-01-23,Washington,53,1,0,7614893,0
3,2020-01-24,Washington,53,1,0,7614893,0
4,2020-01-25,Washington,53,1,0,7614893,0


## JSON Exports

In [85]:
# Master DF to JSON
state_master_df.to_json("../data/state-master-data.json", orient="records")

In [86]:
# NYT Master to JSON 
nyt_master.to_json("../data/nyt-master.json", orient="records")

In [87]:
nyt_avg_daily_cases.to_json("../data/daily_new_cases.json", orient="records")

### CSV Exports

In [88]:
state_master_df.to_csv("../data/state-master-data.csv")

## Amazon RDS Updates

In [89]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [90]:
# 50 States & D.C. Data
state_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [91]:
# Daily Cases Data
nyt_avg_daily_cases.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [ ]:
# Raw NTY Data
nytimes_nih_covid_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)